## Setting up environment

In [1]:
import os
import sys
import warnings

import numpy as np
import tensorflow as tf
import keras
import madmom
import mirdata
import librosa
import random
from sklearn.model_selection import KFold
import json
import pickle
import librosa.display

from scipy.ndimage import maximum_filter1d
from scipy.interpolate import interp1d
from scipy.signal import argrelmax

import matplotlib.pyplot as plt

In [2]:
# ignore certain warnings
warnings.filterwarnings('ignore')

# set default figure size
plt.rcParams['figure.figsize'] = (20, 6)

# Dataset handling

In [3]:
gtzan = mirdata.initialize('gtzan_genre', data_home="./dataset/gtzan_genre")
# gtzan.download()
print("GTZAN", len(gtzan.track_ids))

ballroom = mirdata.initialize('ballroom', data_home="./dataset/ballroom")
# ballroom.download()
print("BALLROOM", len(ballroom.track_ids))

hainsworth = mirdata.initialize('hainsworth', data_home="C:/Users/vanes/Documents/facultate/licenta/project/dataset/hainsworth/H_1.0")
# hainsworth.download()
print("HAINSWORTH", len(hainsworth.track_ids))

giantsteps = mirdata.initialize('giantsteps_tempo', data_home="C:/Users/vanes/Documents/facultate/licenta/project/dataset/giantsteps_tempo")
# giantsteps.download()
print("GIANTSTEPS", len(giantsteps.track_ids))

smc = mirdata.initialize('smc', data_home="C:/Users/vanes/Documents/facultate/licenta/project/dataset/smc")
# giantsteps.download()
print("SMC", len(smc.track_ids))

carnatic = mirdata.initialize('carnatic', data_home="C:/Users/vanes/Documents/facultate/licenta/project/dataset/carnatic")
# carnatic.download()
print("CARNATIC", len(carnatic.track_ids))

harmonix = mirdata.initialize('harmonix', data_home="C:/Users/vanes/Documents/facultate/licenta/project/dataset/harmonix")
# harmonix.download()
print("HARMONIX", len(harmonix.track_ids))

GTZAN 1000
BALLROOM 698
HAINSWORTH 222
GIANTSTEPS 664
SMC 217
CARNATIC 176
HARMONIX 821


## Define dataset splits

* acest lucru l am facut in utils

In [9]:
tracks = {
    "gtzan_genre": gtzan.load_tracks(),
    # "ballroom": ballroom.load_tracks(),
    # "hainsworth": hainsworth.load_tracks(),
    # "smc": smc.load_tracks(),
    # "giantsteps_tempo" : giantsteps.load_tracks(),
    # "carnatic": carnatic.load_tracks(),
    # "harmonix": harmonix.load_tracks()
}

# Preprocesare

In [10]:
from madmom.processors import ParallelProcessor, SequentialProcessor
from madmom.audio.signal import SignalProcessor, FramedSignalProcessor
from madmom.audio.stft import ShortTimeFourierTransformProcessor
from madmom.audio.spectrogram import FilteredSpectrogramProcessor, LogarithmicSpectrogramProcessor

FPS = 100
FFT_SIZE = 2048
NUM_BANDS = 12

# define pre-processor
class PreProcessor(SequentialProcessor):
    def __init__(self, frame_size=FFT_SIZE, num_bands=NUM_BANDS, log=np.log, add=1e-6, fps=FPS):
        # resample to a fixed sample rate in order to get always the same number of filter bins
        sig = SignalProcessor(num_channels=1, sample_rate=44100)
        # split audio signal in overlapping frames
        frames = FramedSignalProcessor(frame_size=frame_size, fps=fps)
        # compute STFT
        stft = ShortTimeFourierTransformProcessor()
        # filter the magnitudes
        filt = FilteredSpectrogramProcessor(num_bands=num_bands)
        # scale them logarithmically
        spec = LogarithmicSpectrogramProcessor(log=log, add=add)
        # instantiate a SequentialProcessor
        super(PreProcessor, self).__init__((sig, frames, stft, filt, spec, np.array))
        # safe fps as attribute (needed for quantization of events)
        self.fps = fps

# Reteau neuronala

In [6]:
import keras.backend as K

from keras.models import Sequential, Model
from keras.layers import (
    Activation,
    Dense,
    Input,
    Conv1D,
    Conv2D,
    MaxPooling2D,
    Reshape,
    Dropout,
    SpatialDropout1D,
    GaussianNoise,
    GlobalAveragePooling1D,
)

# from keras.legacy import interfaces
from keras.utils import Sequence
from keras.optimizers import Optimizer

## Straturi TCN

Componenta principala a retelei este un TCN si este formata din 11 straturi TCN, fiecare avand rate de dilatare din ce in ce mai mari.

Structura este urmatoarea:
<img src="image.png" alt="Structura TCN multi-task" width="400"/>

In [7]:
def residual_block(x, i, activation, num_filters, kernel_size, padding, dropout_rate=0, name=''):
    # name of the layer
    name = name + '_dilation_%d' % i
    # 1x1 conv. of input (so it can be added as residual)
    res_x = Conv1D(num_filters, 1, padding='same', name=name + '_1x1_conv_residual')(x)
    # two dilated convolutions, with dilation rates of i and 2i
    conv_1 = Conv1D(
        filters=num_filters,
        kernel_size=kernel_size,
        dilation_rate=i,
        padding=padding,
        name=name + '_dilated_conv_1',
    )(x)
    conv_2 = Conv1D(
        filters=num_filters,
        kernel_size=kernel_size,
        dilation_rate=i * 2,
        padding=padding,
        name=name + '_dilated_conv_2',
    )(x)
    # concatenate the output of the two dilations
    concat = keras.layers.concatenate([conv_1, conv_2], name=name + '_concat')
    # apply activation function
    x = Activation(activation, name=name + '_activation')(concat)
    # apply spatial dropout
    x = SpatialDropout1D(dropout_rate, name=name + '_spatial_dropout_%f' % dropout_rate)(x)
    # 1x1 conv. to obtain a representation with the same size as the residual
    x = Conv1D(num_filters, 1, padding='same', name=name + '_1x1_conv')(x)
    # add the residual to the processed data and also return it as skip connection
    return keras.layers.add([res_x, x], name=name + '_merge_residual'), x


class TCN:
    def __init__(
        self,
        num_filters=20,
        kernel_size=5,
        dilations=[1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024],
        activation='elu',
        padding='same',
        dropout_rate=0.15,
        name='tcn',
    ):
        self.name = name
        self.dropout_rate = dropout_rate
        self.activation = activation
        self.dilations = dilations
        self.kernel_size = kernel_size
        self.num_filters = num_filters
        self.padding = padding

        if padding != 'causal' and padding != 'same':
            raise ValueError("Only 'causal' or 'same' padding are compatible for this layer.")

    def __call__(self, inputs):
        x = inputs
        # gather skip connections, each having a different context
        skip_connections = []
        # build the TCN models
        for i, num_filters in zip(self.dilations, self.num_filters):
            # feed the output of the previous layer into the next layer
            # increase dilation rate for each consecutive layer
            x, skip_out = residual_block(
                x, i, self.activation, num_filters, self.kernel_size, self.padding, self.dropout_rate, name=self.name
            )
            # collect skip connection
            skip_connections.append(skip_out)
        # activate the output of the TCN stack
        x = Activation(self.activation, name=self.name + '_activation')(x)
        # merge the skip connections by simply adding them
        skip = keras.layers.add(skip_connections, name=self.name + '_merge_skip_connections')
        return x, skip

## Multi-task model

Reteaua care urmeaza sa fie antrenata este formata din doua parti principale:

* un ansamblu de straturi convolutionale si

* TCN-ul propriu-zis

Prima parte este folosita pentru a invata trasaturi locale relevante, in timp ce a doua parte invata dependentele temporale ale acestor trasaturi (locale).

In [9]:
def create_model(input_shape, num_filters=20, num_dilations=11, kernel_size=5, activation='elu', dropout_rate=0.15):
    # input layer
    input_layer = Input(shape=input_shape)

    # stack of 3 conv layers, each conv, activation, max. pooling & dropout
    conv_1 = Conv2D(num_filters, (3, 3), padding='valid', name='conv_1_conv')(input_layer)
    conv_1 = Activation(activation, name='conv_1_activation')(conv_1)
    conv_1 = MaxPooling2D((1, 3), name='conv_1_max_pooling')(conv_1)
    conv_1 = Dropout(dropout_rate, name='conv_1_dropout')(conv_1)

    conv_2 = Conv2D(num_filters, (1, 10), padding='valid', name='conv_2_conv')(conv_1)
    conv_2 = Activation(activation, name='conv_2_activation')(conv_2)
    conv_2 = MaxPooling2D((1, 3), name='conv_2_max_pooling')(conv_2)
    conv_2 = Dropout(dropout_rate, name='conv_2_dropout')(conv_2)

    conv_3 = Conv2D(num_filters, (3, 3), padding='valid', name='conv_3_conv')(conv_2)
    conv_3 = Activation(activation, name='conv_3_activation')(conv_3)
    conv_3 = MaxPooling2D((1, 3), name='conv_3_max_pooling')(conv_3)
    conv_3 = Dropout(dropout_rate, name='conv_3_dropout')(conv_3)

    # reshape layer to reduce dimensions
    x = Reshape((-1, num_filters), name='tcn_input_reshape')(conv_3)

    # TCN layers
    dilations = [2 ** i for i in range(num_dilations)]
    tcn, skip = TCN(
        num_filters=[num_filters] * len(dilations),
        kernel_size=kernel_size,
        dilations=dilations,
        activation=activation,
        padding='same',
        dropout_rate=dropout_rate,
    )(x)

    # output layers; beats & downbeats use TCN output, tempo the skip connections
    beats = Dropout(dropout_rate, name='beats_dropout')(tcn)
    beats = Dense(1, name='beats_dense')(beats)
    beats = Activation('sigmoid', name='beats')(beats)

    downbeats = Dropout(dropout_rate, name='downbeats_dropout')(tcn)
    downbeats = Dense(1, name='downbeats_dense')(downbeats)
    downbeats = Activation('sigmoid', name='downbeats')(downbeats)

    tempo = Dropout(dropout_rate, name='tempo_dropout')(skip)
    tempo = GlobalAveragePooling1D(name='tempo_global_average_pooling')(tempo)
    tempo = GaussianNoise(dropout_rate, name='tempo_noise')(tempo)
    tempo = Dense(300, name='tempo_dense')(tempo)
    tempo = Activation('softmax', name='tempo')(tempo)

    # instantiate a Model and return it
    return Model(input_layer, outputs=[beats, downbeats, tempo])

## DataSequence

Pentru a putea antrena reteaua, trebuie sa furnizam piesele intr-un mod pe care reteaua il poate procesa.

Deoarece modelul trebuie sa poata procesa secvente (adica piese muzicale) de lungime variabila, se va folosi un batch size de 1.
Astfel, nu este nevoie de padding pentru secvente si putem pur si simplu sa iteram peste ele.

Ca trasaturi (features), folosim o reprezentare spectrograma, iar ca tinte (targets) folosim adnotarile beats, downbeats si tempo ale pieselor.
Informatiile despre tempo sunt intotdeauna calculate din beats.

Beats si downbeats sunt codificate one-hot, adica frame-urile care corespund unui downbeat au valoarea 1, iar toate celelalte frame-uri au valoarea 0.

Tempo-ul este codificat ca un vector in care bin-ul corespunzator valorii tinta a tempo-ului (in bpm – batai pe minut) are valoarea 1, iar toate celelalte 0.

Daca nu exista informatii despre downbeats, mascam tintele (toate valorile setate la -1).
In acest fel, eroarea poate fi ignorata si nu se propaga inapoi (backpropagation) la actualizarea ponderilor.

Pentru a imbunatati acuratetea si viteza de antrenare, „largim” tintele, adica oferim frame-urilor / bin-urilor vecine o valoare intre 0 si 1.

In [ ]:
MASK_VALUE = -1

# infer (global) tempo from beats
def infer_tempo(beats, hist_smooth=15, fps=FPS, no_tempo=MASK_VALUE):
    ibis = np.diff(beats) * fps
    bins = np.bincount(np.round(ibis).astype(int))
    # if no beats are present, there is no tempo
    if not bins.any():
        return NO_TEMPO
    intervals = np.arange(len(bins))
    # smooth histogram bins
    if hist_smooth > 0:
        bins = madmom.audio.signal.smooth(bins, hist_smooth)
    # create interpolation function
    interpolation_fn = interp1d(intervals, bins, 'quadratic')
    # generate new intervals with 1000x the resolution
    intervals = np.arange(intervals[0], intervals[-1], 0.001)
    tempi = 60.0 * fps / intervals
    # apply quadratic interpolation
    bins = interpolation_fn(intervals)
    peaks = argrelmax(bins, mode='wrap')[0]
    if len(peaks) == 0:
        # no peaks, no tempo
        return no_tempo
    else:
        # report only the strongest tempo
        sorted_peaks = peaks[np.argsort(bins[peaks])[::-1]]
        return tempi[sorted_peaks][0]


# pad features
def cnn_pad(data, pad_frames):
    """Pad the data by repeating the first and last frame N times."""
    pad_start = np.repeat(data[:1], pad_frames, axis=0)
    pad_stop = np.repeat(data[-1:], pad_frames, axis=0)
    return np.concatenate((pad_start, data, pad_stop))

In [ ]:
# nu permite piese fara adnotari de beat (trece peste ele)
# wrap training/test data as a Keras sequence so we can use it with fit_generator()
class DataSequence(Sequence):
    def __init__(self, tracks, pre_processor, num_tempo_bins=300, pad_frames=None):
        # store features and targets in dictionaries with name of the song as key
        self.x = {}
        self.beats = {}
        self.downbeats = {}
        self.tempo = {}
        self.pad_frames = pad_frames
        self.ids = []
        # iterate over all tracks
        for i, key in enumerate(tracks):
            # print progress
            sys.stderr.write(f'\rprocessing track {i + 1}/{len(tracks)}: {key + " " * 20}')
            sys.stderr.flush()
            t = tracks[key]
            try:
                # use track only if it contains beats
                beats = t.beats.times
                # wrap librosa wav data & sample rate as Signal
                s = madmom.audio.Signal(*t.audio)
                # compute features first to be able to quantize beats
                x = pre_processor(s)
                self.x[key] = x
                # quantize beats
                beats = madmom.utils.quantize_events(beats, fps=pre_processor.fps, length=len(x))
                self.beats[key] = beats
            except AttributeError as e:
                # no beats found, skip this file
                print(f'\r{key} has no beat information, skipping\n')
                print(e)
                continue
            # downbeats
            try:
                downbeats = t.beats.positions.astype(int) == 1
                downbeats = t.beats.times[downbeats]
                downbeats = madmom.utils.quantize_events(downbeats, fps=pre_processor.fps, length=len(x))
            except AttributeError:
                print(f'\r{key} has no downbeat information, masking\n')
                downbeats = np.ones(len(x), dtype='float32') * MASK_VALUE
            self.downbeats[key] = downbeats
            # tempo
            tempo = None
            try:
                # Note: to be able to augment a dataset, we need to scale the beat times
                tempo = infer_tempo(t.beats.times * pre_processor.fps / 100, fps=pre_processor.fps)
                tempo = keras.utils.to_categorical(int(np.round(tempo)), num_classes=num_tempo_bins, dtype='float32')
            except IndexError:
                # tempo out of bounds (too high)
                print(f'\r{key} has no valid tempo ({tempo}), masking\n')
                tempo = np.ones(num_tempo_bins, dtype='float32') * MASK_VALUE
            self.tempo[key] = tempo
            # keep track of IDs
            self.ids.append(key)
        assert len(self.x) == len(self.beats) == len(self.downbeats) == len(self.tempo) == len(self.ids)

    def __len__(self):
        return len(self.ids)

    def __getitem__(self, idx):
        # convert int idx to key
        if isinstance(idx, int):
            idx = self.ids[idx]
        # Note: we always use a batch size of 1 since the tracks have variable length
        #       keras expects the batch to be the first dimension, the prepend an axis;
        #       append an axis to beats and downbeats as well
        # define targets
        y = {}
        y['beats'] = self.beats[idx][np.newaxis, ..., np.newaxis]
        y['downbeats'] = self.downbeats[idx][np.newaxis, ..., np.newaxis]
        y['tempo'] = self.tempo[idx][np.newaxis, ...]
        # add context to frames
        x = self.x[idx]
        if self.pad_frames:
            x = cnn_pad(x, self.pad_frames)
        return x[np.newaxis, ..., np.newaxis], y

    def widen_beat_targets(self, size=3, value=0.5):
        for y in self.beats.values():
            # skip masked beat targets
            if np.allclose(y, MASK_VALUE):
                continue
            np.maximum(y, maximum_filter1d(y, size=size) * value, out=y)

    def widen_downbeat_targets(self, size=3, value=0.5):
        for y in self.downbeats.values():
            # skip masked downbeat targets
            if np.allclose(y, MASK_VALUE):
                continue
            np.maximum(y, maximum_filter1d(y, size=size) * value, out=y)

    def widen_tempo_targets(self, size=3, value=0.5):
        for y in self.tempo.values():
            # skip masked tempo targets
            if np.allclose(y, MASK_VALUE):
                continue
            np.maximum(y, maximum_filter1d(y, size=size) * value, out=y)

    def append(self, other):
        assert not any(key in self.ids for key in other.ids), 'IDs must be unique'
        self.x.update(other.x)
        self.beats.update(other.beats)
        self.downbeats.update(other.downbeats)
        self.tempo.update(other.tempo)
        self.ids.extend(other.ids)

In [ ]:
# permite si beat uri lipsa
class DataSequence(Sequence):
    def __init__(self, tracks, pre_processor, num_tempo_bins=300, pad_frames=None):
        self.x = {}
        self.beats = {}
        self.downbeats = {}
        self.tempo = {}
        self.pad_frames = pad_frames
        self.ids = []

        for i, key in enumerate(tracks):
            sys.stderr.write(f'\rprocessing track {i + 1}/{len(tracks)}: {key + " " * 20}')
            sys.stderr.flush()
            t = tracks[key]

            try:
                s = madmom.audio.Signal(*t.audio)
                x = pre_processor(s)
                self.x[key] = x
                length = len(x)
            except Exception as e:
                print(f"\nCould not process audio for {key}: {e}")
                continue

            # beats
            try:
                beats = t.beats.times
                beats = madmom.utils.quantize_events(beats, fps=pre_processor.fps, length=length)
            except AttributeError:
                beats = np.ones(length, dtype='float32') * MASK_VALUE
            self.beats[key] = beats

            # downbeats
            try:
                downbeats = t.beats.positions.astype(int) == 1
                downbeats = t.beats.times[downbeats]
                downbeats = madmom.utils.quantize_events(downbeats, fps=pre_processor.fps, length=length)
            except AttributeError:
                downbeats = np.ones(length, dtype='float32') * MASK_VALUE
            self.downbeats[key] = downbeats

            # tempo
            try:
                tempo = infer_tempo(t.beats.times * pre_processor.fps / 100, fps=pre_processor.fps)
                tempo = keras.utils.to_categorical(int(np.round(tempo)), num_classes=num_tempo_bins, dtype='float32')
            except Exception:
                tempo = np.ones(num_tempo_bins, dtype='float32') * MASK_VALUE
            self.tempo[key] = tempo

            self.ids.append(key)

        assert len(self.x) == len(self.beats) == len(self.downbeats) == len(self.tempo) == len(self.ids)

    def __len__(self):
        return len(self.ids)

    def __getitem__(self, idx):
        if isinstance(idx, int):
            idx = self.ids[idx]

        y = {
            'beats': self.beats[idx][np.newaxis, ..., np.newaxis],
            'downbeats': self.downbeats[idx][np.newaxis, ..., np.newaxis],
            'tempo': self.tempo[idx][np.newaxis, ...]
        }

        x = self.x[idx]
        if self.pad_frames:
            x = cnn_pad(x, self.pad_frames)
        return x[np.newaxis, ..., np.newaxis], y
    
    def widen_beat_targets(self, size=3, value=0.5):
        for y in self.beats.values():
            # skip masked beat targets
            if np.allclose(y, MASK_VALUE):
                continue
            np.maximum(y, maximum_filter1d(y, size=size) * value, out=y)

    def widen_downbeat_targets(self, size=3, value=0.5):
        for y in self.downbeats.values():
            # skip masked downbeat targets
            if np.allclose(y, MASK_VALUE):
                continue
            np.maximum(y, maximum_filter1d(y, size=size) * value, out=y)

    def widen_tempo_targets(self, size=3, value=0.5):
        for y in self.tempo.values():
            # skip masked tempo targets
            if np.allclose(y, MASK_VALUE):
                continue
            np.maximum(y, maximum_filter1d(y, size=size) * value, out=y)

    def append(self, other):
        assert not any(key in self.ids for key in other.ids), 'IDs must be unique'
        self.x.update(other.x)
        self.beats.update(other.beats)
        self.downbeats.update(other.downbeats)
        self.tempo.update(other.tempo)
        self.ids.extend(other.ids)


## Loss si metrici

Antrenăm modelul folosind funcția de pierdere cross entropy, însă trebuie să mascăm această funcție de pierdere și metricele asociate în cazul în care adnotarile sunt mascate.

In [ ]:
# https://github.com/keras-team/keras/issues/3893
def build_masked_loss(loss_function, mask_value=MASK_VALUE):
    """Builds a loss function that masks based on targets

    Args:
        loss_function: The loss function to mask
        mask_value: The value to mask in the targets

    Returns:
        function: a loss function that acts like loss_function with masked inputs
    """

    def masked_loss_function(y_true, y_pred):
        mask = K.cast(K.not_equal(y_true, mask_value), K.floatx())
        return loss_function(y_true * mask, y_pred * mask)

    return masked_loss_function

# Train network

## Training & testing sequences

We wrap our previously split dataset as `DataSequences`.

We widen the beat and downbeat targets to have a value of 0.5 at the frames next to the annotated beat locations.

We assign tempo values ±1 bpm apart a value of 0.5, and those ±2bpm a value of0.25.

In [ ]:
# pentru eficienta am salvat o singura data DataSequence cu toate si apoi am facut foldurile (tot DataSequence)

# Preprocesor
pre_processor = PreProcessor()

all_tracks = {}
for dataset_name, dataset_tracks in tracks.items():
    for track_id, track in dataset_tracks.items():
        all_tracks[(dataset_name, track_id)] = track

sequence = DataSequence(
    {f"{ds}_{tid}": t for (ds, tid), t in all_tracks.items()},
    pre_processor=pre_processor,
    # num_tempo_bins=300,
    pad_frames=2
)

with open("fold_all.pkl", "wb") as f:
    pickle.dump(sequence, f)


In [ ]:
with open("data_sequence_all.pkl", "rb") as f:
    full_sequence = pickle.load(f)

In [ ]:
# Preprocesor
pre_processor = PreProcessor()

os.makedirs("folds_all", exist_ok=True)

# Încarcă folds.json
with open("folds_all.json", "r") as f:
    folds = json.load(f)

for i, fold in enumerate(folds):
    for split in ['train','test']:
        split_ids = [f"{dataset}_{track_id}" for dataset, track_id in fold[split] if f"{dataset}_{track_id}" in full_sequence.ids]
        
        split_seq = DataSequence({}, pre_processor=pre_processor, pad_frames=2)
        for track_id in split_ids:
            split_seq.x[track_id] = full_sequence.x[track_id]
            split_seq.beats[track_id] = full_sequence.beats[track_id]
            split_seq.downbeats[track_id] = full_sequence.downbeats[track_id]
            split_seq.tempo[track_id] = full_sequence.tempo[track_id]
            split_seq.ids.append(track_id)

        with open(f"folds_giantsteps/fold_{i}_{split}.pkl", "wb") as f_out:
            pickle.dump(split_seq, f_out)

        print(f"Salvat: fold_{i}_{split}.pkl ({len(split_seq)} piese)")

In [ ]:
with open("./folds_gtzan/fold_8_train.pkl", "rb") as f:
    train = pickle.load(f)

with open("./folds_gtzan/fold_8_test.pkl", "rb") as f:
    test = pickle.load(f) 

In [ ]:
train.widen_beat_targets()
train.widen_downbeat_targets()
train.widen_tempo_targets()
train.widen_tempo_targets()

test.widen_beat_targets()
test.widen_downbeat_targets()
test.widen_tempo_targets()
test.widen_tempo_targets()

## Visualizam training data-ul

In [ ]:
# cum arata spectrograma reala (modificata putin fta de cea reala)
# pentru primele 5 piese din train

DATASETS = [
    "gtzan_genre", "ballroom", "hainsworth",
    "giantsteps_tempo", "smc", "carnatic", "harmonix"
]

sr = 44100
fps = 100
hop_duration = 1.0 / fps

for i in range(min(5, len(train))):
    plt.clf()      # Clear current figure
    plt.cla()      # Clear current axes
    plt.close()    # Close the figure window (de obicei opțional)

    x, y = train[i]
    full_id = train.ids[i]
    print(f"\nTrack {i} - ID complet: {full_id}")

    dataset = next((d for d in DATASETS if full_id.startswith(f"{d}_")), None)
    if dataset is None:
        print("Dataset necunoscut")
        continue

    track_id = full_id[len(dataset) + 1:]
    
    try:
        track = tracks[dataset][track_id]
    except KeyError:
        print(f"Track ID {track_id} nu există în datasetul {dataset}.")
        continue

    spec = x[0, :, :, 0].T
    n_frames = spec.shape[1]
    duration_sec = n_frames * hop_duration

    fig_width = (duration_sec / 30) * 12
    plt.figure(figsize=(fig_width, 4))
    plt.imshow(spec, aspect='auto', origin='lower', cmap='magma',
               extent=[0, duration_sec, 0, fps / 2])

    try:
        beat_times = track.beats.times
        plt.vlines(beat_times, 0, fps / 2, linestyles='dotted', color='w', linewidth=0.9)
    except AttributeError:
        print("Beats indisponibili")

    try:
        downbeat_times = track.beats.times[track.beats.positions == 1]
        plt.vlines(downbeat_times, 0, fps / 2, color='w', linewidth=0.8)
    except (AttributeError, IndexError):
        print("Downbeats indisponibili")

    plt.title(f"Spectrograma (madmom PreProcessor) – {full_id}")
    plt.xlabel("Timp (secunde)")
    plt.ylabel("Benzi log (nu Hz)")
    plt.colorbar(label="Log-energie")
    plt.tight_layout()
    plt.show()


In [ ]:
# cum arata ea de obicei

song = train.ids[4]
print(f"Song ID: {song}")

DATASETS = [
    "gtzan_genre", "ballroom", "hainsworth",
    "giantsteps_tempo", "smc", "carnatic", "harmonix"
]

dataset = next((d for d in DATASETS if song.startswith(f"{d}_")), None)

if dataset:
    print(f"Song provine din datasetul: {dataset}")
    track_id = song[len(dataset) + 1:]  # elimină prefixul și "_" ca să rămână doar ID-ul
    track = tracks[dataset][track_id]
else:
    raise ValueError("Piesa nu apartine unui dataset cunoscut")

audio, sr = track.audio
hop_length = 512

duration_sec = len(audio) / sr

fig_width = (duration_sec / 30) * 12

plt.figure(figsize=(fig_width, 4))
spec = librosa.amplitude_to_db(np.abs(librosa.stft(audio, hop_length=hop_length)), ref=np.max)
img = librosa.display.specshow(spec, y_axis='log', sr=sr, hop_length=hop_length, x_axis='time')

try:
    beat_times = track.beats.times
    plt.vlines(beat_times, 0, sr / 2, linestyles='dotted', color='w', linewidth=0.9)
except AttributeError:
    print("Nu are beat-uri")

try:
    downbeat_times = track.beats.times[track.beats.positions == 1]
    plt.vlines(downbeat_times, 0, sr / 2, color='w', linewidth=0.8)
except (AttributeError, IndexError):
    print("Nu are downbeat-uri")

plt.title(f'Log-frequency power spectrogram of track: "{song}"')
plt.colorbar(format="%+2.f dB")
plt.show()

## Putem modelul cap la cap

In [ ]:
from tensorflow.keras.optimizers import Adam
import tensorflow_addons as tfa
from tensorflow.keras.callbacks import CSVLogger

# use a training sample to infer input shape
input_shape = (None,) + train[0][0].shape[-2:]
model = create_model(input_shape)

learnrate = 0.005
clipnorm = 0.5

optimizer = tfa.optimizers.RectifiedAdam(learning_rate=learnrate,clipnorm=clipnorm)
model.compile(
    optimizer=optimizer,
    loss=[
        build_masked_loss(K.binary_crossentropy),
        build_masked_loss(K.binary_crossentropy),
        build_masked_loss(K.binary_crossentropy),
    ],
    metrics=['binary_accuracy'],
)
model.summary(200)

Model: "model"
________________________________________________________________________________________________________________________________________________________________________________________________________
 Layer (type)                                                     Output Shape                                Param #                 Connected to                                                      
 input_1 (InputLayer)                                             [(None, None, 81, 1)]                       0                       []                                                                
                                                                                                                                                                                                        
 conv_1_conv (Conv2D)                                             (None, None, 79, 20)                        200                     ['input_1[0][0]']                              

## Antrenare

In [ ]:
setting = 'multitask_fold_0_1.5'
epochs = 100
verbose = 0

PATH = '.'

# create output dir
outdir = f'{PATH}/models/{setting}'
os.makedirs(outdir, exist_ok=True)
print(f'output dir: {outdir}')

output dir: ./models_good/models/multitask_fold_0_1.5


In [23]:
csv_logger = CSVLogger(f'{outdir}/logs.csv', append=False)

# model checkpointing
mc = keras.callbacks.ModelCheckpoint(f'{outdir}/model_best.h5', monitor='loss', save_best_only=True, verbose=verbose)

# learn rate scheduler
lr = keras.callbacks.ReduceLROnPlateau(
    monitor='loss', factor=0.2, patience=10, verbose=1, mode='auto', min_delta=1e-3, cooldown=0, min_lr=1e-7
)

# early stopping
es = keras.callbacks.EarlyStopping(monitor='loss', min_delta=1e-4, patience=20, verbose=verbose)

# tensorboard logging
tb = keras.callbacks.TensorBoard(log_dir=f'{outdir}/logs', write_graph=True, write_images=True)

# actually train network
history = model.fit_generator(
    train,
    steps_per_epoch=len(train),
    epochs=epochs,
    shuffle=True,
    validation_data=test,
    validation_steps=len(test),
    callbacks=[mc, es, tb, lr, csv_logger],
)
model.save(f'{outdir}/model_final.h5')

Epoch 1/100
874/874 [==============================] - 147s 158ms/step - loss: 0.3298 - beats_loss: 0.1908 - downbeats_loss: 0.0893 - tempo_loss: 0.0497 - beats_binary_accuracy: 0.9342 - downbeats_binary_accuracy: 0.9731 - tempo_binary_accuracy: 0.9822 - val_loss: 0.2026 - val_beats_loss: 0.1157 - val_downbeats_loss: 0.0427 - val_tempo_loss: 0.0442 - val_beats_binary_accuracy: 0.9440 - val_downbeats_binary_accuracy: 0.9779 - val_tempo_binary_accuracy: 0.9833 - lr: 0.0050
Epoch 2/100
874/874 [==============================] - 156s 178ms/step - loss: 0.2267 - beats_loss: 0.1313 - downbeats_loss: 0.0503 - tempo_loss: 0.0450 - beats_binary_accuracy: 0.9393 - downbeats_binary_accuracy: 0.9786 - tempo_binary_accuracy: 0.9822 - val_loss: 0.1932 - val_beats_loss: 0.1134 - val_downbeats_loss: 0.0423 - val_tempo_loss: 0.0375 - val_beats_binary_accuracy: 0.9441 - val_downbeats_binary_accuracy: 0.9779 - val_tempo_binary_accuracy: 0.9833 - lr: 0.0050
Epoch 3/100
874/874 [===========================

# Post-procesare

In [ ]:
# track beats with a DBN
np.int = int
np.float = float
beat_tracker = madmom.features.beats.DBNBeatTrackingProcessor(
    min_bpm=55.0, max_bpm=215.0, fps=FPS, transition_lambda=100, threshold=0.05
)

# track downbeats with a DBN
# as input, use a combined beat & downbeat activation function
downbeat_tracker = madmom.features.downbeats.DBNDownBeatTrackingProcessor(
    beats_per_bar=[3, 4], min_bpm=55.0, max_bpm=215.0, fps=FPS, transition_lambda=100,
)

# track bars, i.e. first track the beats and then infer the downbeat positions
bar_tracker = madmom.features.downbeats.DBNBarTrackingProcessor(
    beats_per_bar=(3, 4), meter_change_prob=1e-3, observation_weight=4,
)

In [43]:
def detect_tempo(bins, hist_smooth=11, min_bpm=10):
    min_bpm = int(np.floor(min_bpm))
    tempi = np.arange(min_bpm, len(bins))
    bins = bins[min_bpm:]
    # smooth histogram bins
    if hist_smooth > 0:
        bins = madmom.audio.signal.smooth(bins, hist_smooth)
    # create interpolation function
    interpolation_fn = interp1d(tempi, bins, 'quadratic')
    # generate new intervals with 1000x the resolution
    tempi = np.arange(tempi[0], tempi[-1], 0.001)
    # apply quadratic interpolation
    bins = interpolation_fn(tempi)
    peaks = argrelmax(bins, mode='wrap')[0]
    if len(peaks) == 0:
        # no peaks, no tempo
        tempi = np.array([], ndmin=2)
    elif len(peaks) == 1:
        # report only the strongest tempo
        ret = np.array([tempi[peaks[0]], 1.0])
        tempi = np.array([tempi[peaks[0]], 1.0])
    else:
        # sort the peaks in descending order of bin heights
        sorted_peaks = peaks[np.argsort(bins[peaks])[::-1]]
        # normalize their strengths
        strengths = bins[sorted_peaks]
        strengths /= np.sum(strengths)
        # return the tempi and their normalized strengths
        ret = np.array(list(zip(tempi[sorted_peaks], strengths)))
        tempi = np.array(list(zip(tempi[sorted_peaks], strengths)))
    return tempi[:2]

In [ ]:
def predict(model, dataset, detdir=None, activations={}, detections={}):
    for i, t in enumerate(dataset):
        f = dataset.ids[i]
        sys.stderr.write('\rprocessing file %d of %d: %12s' % (i + 1, len(dataset), f))
        sys.stderr.flush()

        x = t[0]

        beats, downbeats, tempo = model.predict(x)

        beats_act = beats.squeeze()
        downbeats_act = downbeats.squeeze()
        tempo_act = tempo.squeeze()

        # plt.clf()
        # plt.plot(beats_act_smooth)
        # plt.title("Beat activations")
        # plt.xlabel("Frame")
        # plt.ylabel("Activation")
        # plt.show()

        beats = beat_tracker(beats_act)

        # Compute combined activations
        assert beats_act.shape == downbeats_act.shape, "Activations must have same shape"
        beat_prob = np.maximum(beats_act - downbeats_act, 0)
        downbeat_prob = downbeats_act

        combined_act = np.column_stack((beat_prob, downbeat_prob))

        # print(type(combined_act), combined_act.shape)
        # print(combined_act.dtype)
        for i, row in enumerate(combined_act):
            if not (np.isscalar(row[0]) and np.isscalar(row[1])):
                print(f"Row {i} has non-scalar elements: {row}, types: {type(row[0])}, {type(row[1])}")
                break

        # Run downbeat tracker safely
        try:
            downbeats = downbeat_tracker(combined_act)
        except Exception as e:
            print(f"\nError in downbeat tracking for file {f}: {e}")
            continue


        # bar tracking
        try:
            beat_idx = (beats * FPS).astype(int)

            #verficare
            try:
                if beats.size == 0:
                    raise ValueError("no beats detected")
                beat_idx = (beats * FPS).astype(int)
                if beat_idx.size == 0 or np.any(beat_idx < 0) or np.any(beat_idx >= downbeats_act.shape[0]):
                    raise ValueError("invalid beat indices")
                
                bar_act = maximum_filter1d(downbeats_act, size=3)
                bar_act = bar_act[beat_idx]
                bar_act = np.vstack((beats, bar_act)).T
                bars = bar_tracker(bar_act)

            except Exception as e:
                print(f"\nError in bar tracking for file {f}: {e}")
                bars = np.empty((0, 2))

            bar_act = maximum_filter1d(downbeats_act, size=3)
            bar_act = bar_act[beat_idx]
            bar_act = np.vstack((beats, bar_act)).T
            bars = bar_tracker(bar_act)
        except Exception as e:
            print(f"\nError in bar tracking for file {f}: {e}")
            bars = np.empty((0, 2))

        # tempo detection
        tempo = detect_tempo(tempo_act)

        activations[f] = {
            'beats': beats_act,
            'downbeats': downbeats_act,
            'combined': combined_act,
            'tempo': tempo_act
        }

        detections[f] = {
            'beats': beats,
            'downbeats': downbeats,
            'bars': bars,
            'tempo': tempo
        }

        # save
        if detdir is not None:
            save_path = os.path.join(detdir, f)
            os.makedirs(os.path.dirname(save_path), exist_ok=True)

            np.save(f'{save_path}.beats.npy', beats_act)
            np.save(f'{save_path}.downbeats.npy', downbeats_act)
            np.save(f'{save_path}.tempo.npy', tempo_act)

            madmom.io.write_beats(beats, f'{save_path}.beats.txt')
            madmom.io.write_beats(downbeats, f'{save_path}.downbeats.txt')
            madmom.io.write_beats(bars, f'{save_path}.bars.txt')
            madmom.io.write_tempo(tempo, f'{save_path}.bpm.txt')


    return activations, detections


### Predict

Facem load la model mai intai

In [ ]:
# if training did not finish, but was almost there, use `model_best.h5` instead of `model_final.h5`
# model = keras.models.load_model(f'{outdir}/model_best.h5', compile=False)
# from your_module import masked_loss_function  # sau definește-o din nou aici dacă nu o ai
from tensorflow_addons.optimizers import RectifiedAdam

setting = 'multitask_fold_0_1.3_all5'
# setting = 'multitask_small_gtzan_small'

PATH = '.'

# create output dir
outdir = f'{PATH}/models/{setting}'

masked_loss_function = build_masked_loss(K.binary_crossentropy, MASK_VALUE)
model = keras.models.load_model(
    f'{outdir}/model_best.h5',
    custom_objects={
        'masked_loss_function': masked_loss_function,
        'RectifiedAdam': RectifiedAdam
    },
    compile=True
)


Predict the model's activations (i.e. the raw outputs) and the post-processed detections.

In [45]:
# create a directory to put the activations and detections into
detdir = f'{outdir}/detections/'

# we can use validation set as test set, since we did not use it for anything besides monitoring progress
activations, detections = predict(model, test, detdir)

processing file 1 of 1000: gtzan_genre_classical.00000

1/1 [==============================] - 0s 49ms/step


processing file 2 of 1000: gtzan_genre_classical.00001

1/1 [==============================] - 0s 52ms/step


processing file 3 of 1000: gtzan_genre_classical.00002

...

1/1 [==============================] - 0s 40ms/step


processing file 999 of 1000: gtzan_genre_metal.00098

1/1 [==============================] - 0s 40ms/step


processing file 1000 of 1000: gtzan_genre_metal.00099

1/1 [==============================] - 0s 33ms/step


# Evaluare

In [ ]:
def strip_prefix(key):
    known_prefixes = ["gtzan_genre", "ballroom", "hainsworth", "smc", "carnatic", "giantsteps_tempo"]
    for prefix in known_prefixes:
        if key.startswith(prefix + "_"):
            return key[len(prefix) + 1:]
    return key

# functii de evaluare
def evaluate_beats(detections, annotations):
    evals = []
    for key, det in detections.items():
        if key not in annotations:
            print(f"Lipsa beat annotation pentru {key}")
            continue
        ann = annotations[key]
        e = madmom.evaluation.beats.BeatEvaluation(det, ann)
        evals.append(e)
    return madmom.evaluation.beats.BeatMeanEvaluation(evals)

def evaluate_downbeats(detections, annotations):
    evals = []
    for key, det in detections.items():
        if key not in annotations:
            print(f"Lipsa downbeat annotation pentru {key}")
            continue
        ann = annotations[key]
        e = madmom.evaluation.beats.BeatEvaluation(det, ann, downbeats=True)
        evals.append(e)
    return madmom.evaluation.beats.BeatMeanEvaluation(evals)

def evaluate_tempo(detections, annotations):
    evals = []
    for key, det in detections.items():
        if key not in annotations:
            print(f"Lipsa tempo annotation pentru {key}")
            continue
        ann = annotations[key]
        e = madmom.evaluation.tempo.TempoEvaluation(det, ann)
        evals.append(e)
    return madmom.evaluation.tempo.TempoMeanEvaluation(evals)

In [ ]:
np.bool = bool

beat_detections = {strip_prefix(k): v['beats'] for k, v in detections.items()}
downbeat_detections = {strip_prefix(k): v['downbeats'] for k, v in detections.items()}
bar_detections = {strip_prefix(k): v['bars'] for k, v in detections.items()}
tempo_detections = {strip_prefix(k): v['tempo'][0, 0] for k, v in detections.items()}

# print(beat_detections)
# print(downbeat_detections)
# print(bar_detections)
# print(tempo_detections)

beat_annotations = {}
downbeat_annotations = {}
tempo_annotations = {}

for dataset_name, dataset_tracks in tracks.items():
    for track_id, track in dataset_tracks.items():
        key = track_id 
        # print(key)
        # Tempo
        if hasattr(track, "tempo") and track.tempo is not None:
            tempo_annotations[key] = track.tempo
        # Beats & downbeats
        if hasattr(track, "beats") and track.beats is not None:
            beat_annotations[key] = track.beats.times
            if hasattr(track.beats, "positions"):
                downbeat_annotations[key] = track.beats.times[track.beats.positions == 1]

        # print(tempo_annotations)
        # # print(beat_detections[key])
        # print(downbeat_annotations)
        # print(beat_annotations)

# Evaluari
print(' Beat evaluation\n--------------------')
print(' Beat tracker:    ', evaluate_beats(beat_detections, beat_annotations))
print(' Downbeat tracker:', evaluate_beats(downbeat_detections, beat_annotations))

print('\n Downbeat evaluation\n-----------------------')
print(' Bar tracker:     ', evaluate_downbeats(bar_detections, downbeat_annotations))
print(' Downbeat tracker:', evaluate_downbeats(downbeat_detections, downbeat_annotations))

print('\n Tempo evaluation\n--------------------')
print(' Tempo:           ', evaluate_tempo(tempo_detections, tempo_annotations))